# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse   Shipments  \
0  May 18 2022  3:44PM  237581        18     8036453   
1  May 18 2022  3:33PM  237579        10  0085679119   
2  May 18 2022  3:33PM  237579        10  0085679118   
3  May 18 2022  3:33PM  237579        10  0085679141   
4  May 18 2022  3:33PM  237579        10  0085679140   
5  May 18 2022  3:33PM  237579        10  0085679179   
6  May 18 2022  3:33PM  237579        10  0085682034   
7  May 18 2022  3:33PM  237579        10  0085682617   
8  May 18 2022  3:33PM  237579        10  0085682649   
9  May 18 2022  3:33PM  237579        10  0085682650   

                    Customer ShipmentStatus  
0  Blinc International, Inc.       Released  
1          ISDIN Corporation       Released  
2          ISDIN Corporation       Released  
3          ISDIN Corporation       Released  
4          ISDIN Corporation       Released  
5          ISDIN Corporation       Released  
6          ISDIN Corporation       Released  
7          ISDIN Corporation       Released  
8          ISDIN Corporation       Released  
9          ISDIN Corporation       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
41  237576       Released          1
42  237577       Released         38
43  237578       Released         21
44  237579       Released         10
45  237581       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
237576                NaN        NaN       1.0
237577                NaN        NaN      38.0
237578                NaN        NaN      21.0
237579                NaN        NaN      10.0
237581                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
237448                0.0        0.0       1.0
237458                0.0        0.0       1.0
237478                0.0        0.0      45.0
237492                1.0        2.0       2.0
237507               13.0        3.0       0.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
237448                  0          0         1
237458                  0          0         1
237478                  0          0        45
237492                  1          2         2
237507                 13          3         0

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               237448          0          0         1
1               237458          0          0         1
2               237478          0          0        45
3               237492          1          2         2
4               237507         13          3         0

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               237448                            1
1               237458                            1
2               237478                           45
3               237492         1         2        2
4               237507        13         3

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                   Customer
0    May 18 2022  3:44PM  237581        18  Blinc International, Inc.
1    May 18 2022  3:33PM  237579        10          ISDIN Corporation
11   May 18 2022  3:11PM  237577        10          ISDIN Corporation
49   May 18 2022  3:11PM  237574        10          ISDIN Corporation
84   May 18 2022  3:08PM  237578        10          ISDIN Corporation
105  May 18 2022  3:06PM  237576        21          NBTY Global, Inc.
106  May 18 2022  3:06PM  237575        21          NBTY Global, Inc.
107  May 18 2022  2:55PM  237572        16      ACG North America LLC
108  May 18 2022  2:54PM  237571        22          NBTY Global, Inc.
109  May 18 2022  2:53PM  237570        22          NBTY Global, Inc.

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                   Customer Completed  \
0  May 18 2022  3:44PM  237581        18  Blinc International, Inc.             
1  May 18 2022  3:33PM  237579        10          ISDIN Corporation             
2  May 18 2022  3:11PM  237577        10          ISDIN Corporation             
3  May 18 2022  3:11PM  237574        10          ISDIN Corporation             
4  May 18 2022  3:08PM  237578        10          ISDIN Corporation             
5  May 18 2022  3:06PM  237576        21          NBTY Global, Inc.             
6  May 18 2022  3:06PM  237575        21          NBTY Global, Inc.             
7  May 18 2022  2:55PM  237572        16      ACG North America LLC             
8  May 18 2022  2:54PM  237571        22          NBTY Global, Inc.             
9  May 18 2022  2:53PM  237570        22          NBTY Global, Inc.             

  Executing Released  
0                  1  
1                 10  
2                 38  
3                 35  
4                 21  
5                  1  
6                  1  
7                  1  
8                  1  
9                  1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                   Customer Released  \
0  May 18 2022  3:44PM  237581        18  Blinc International, Inc.        1   
1  May 18 2022  3:33PM  237579        10          ISDIN Corporation       10   
2  May 18 2022  3:11PM  237577        10          ISDIN Corporation       38   
3  May 18 2022  3:11PM  237574        10          ISDIN Corporation       35   
4  May 18 2022  3:08PM  237578        10          ISDIN Corporation       21   
5  May 18 2022  3:06PM  237576        21          NBTY Global, Inc.        1   
6  May 18 2022  3:06PM  237575        21          NBTY Global, Inc.        1   
7  May 18 2022  2:55PM  237572        16      ACG North America LLC        1   
8  May 18 2022  2:54PM  237571        22          NBTY Global, Inc.        1   
9  May 18 2022  2:53PM  237570        22          NBTY Global, Inc.        1   

  Executing Completed  
0                      
1                      
2                      
3                      
4                      
5                      
6                      
7                      
8                      
9

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                   Customer Released  \
0  May 18 2022  3:44PM  237581        18  Blinc International, Inc.        1   
1  May 18 2022  3:33PM  237579        10          ISDIN Corporation       10   
2  May 18 2022  3:11PM  237577        10          ISDIN Corporation       38   
3  May 18 2022  3:11PM  237574        10          ISDIN Corporation       35   
4  May 18 2022  3:08PM  237578        10          ISDIN Corporation       21   

  Executing Completed  
0                      
1                      
2                      
3                      
4

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse                   Customer  Released  \
0  May 18 2022  3:44PM  237581        18  Blinc International, Inc.       1.0   
1  May 18 2022  3:33PM  237579        10          ISDIN Corporation      10.0   
2  May 18 2022  3:11PM  237577        10          ISDIN Corporation      38.0   
3  May 18 2022  3:11PM  237574        10          ISDIN Corporation      35.0   
4  May 18 2022  3:08PM  237578        10          ISDIN Corporation      21.0   

   Executing  Completed  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                   Customer  Released  \
0  May 18 2022  3:44PM  237581        18  Blinc International, Inc.       1.0   
1  May 18 2022  3:33PM  237579        10          ISDIN Corporation      10.0   
2  May 18 2022  3:11PM  237577        10          ISDIN Corporation      38.0   
3  May 18 2022  3:11PM  237574        10          ISDIN Corporation      35.0   
4  May 18 2022  3:08PM  237578        10          ISDIN Corporation      21.0   

   Executing  Completed  
0        0.0        0.0  
1        0.0        0.0  
2        0.0        0.0  
3        0.0        0.0  
4        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         3563210     206.0       17.0        2.0
12          237541       0.0        1.0        0.0
16          712651      12.0       12.0        0.0
18         1425127       8.0        3.0       13.0
19          712585       3.0        3.0        1.0
20          950134      30.0        1.0        0.0
21          475151       2.0        0.0        0.0
22          475141       2.0        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  3563210     206.0       17.0        2.0
1        12   237541       0.0        1.0        0.0
2        16   712651      12.0       12.0        0.0
3        18  1425127       8.0        3.0       13.0
4        19   712585       3.0        3.0        1.0
5        20   950134      30.0        1.0        0.0
6        21   475151       2.0        0.0        0.0
7        22   475141       2.0        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10     206.0       17.0        2.0
1        12       0.0        1.0        0.0
2        16      12.0       12.0        0.0
3        18       8.0        3.0       13.0
4        19       3.0        3.0        1.0
5        20      30.0        1.0        0.0
6        21       2.0        0.0        0.0
7        22       2.0        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released  206.0
1        12  Released    0.0
2        16  Released   12.0
3        18  Released    8.0
4        19  Released    3.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse     10   12    16    18   19    20   21   22
Status                                                
Completed    2.0  0.0   0.0  13.0  1.0   0.0  0.0  0.0
Executing   17.0  1.0  12.0   3.0  3.0   1.0  0.0  0.0
Released   206.0  0.0  12.0   8.0  3.0  30.0  2.0  2.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status     10   12    16    18   19    20   21   22
0          Completed    2.0  0.0   0.0  13.0  1.0   0.0  0.0  0.0
1          Executing   17.0  1.0  12.0   3.0  3.0   1.0  0.0  0.0
2           Released  206.0  0.0  12.0   8.0  3.0  30.0  2.0  2.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status     10   12    16    18   19    20   21   22
0  Completed    2.0  0.0   0.0  13.0  1.0   0.0  0.0  0.0
1  Executing   17.0  1.0  12.0   3.0  3.0   1.0  0.0  0.0
2   Released  206.0  0.0  12.0   8.0  3.0  30.0  2.0  2.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()